# MTH3302 - Méthodes probabilistes et statistiques pour I.A.
#### Polytechnique Montréal
### Projet A2024
----
Équipe T - TODO
### Objectif
Prédiction de **la consommation en carburant de voitures récentes**.

### Données
Le jeu de données contient pour presque 400 véhicule, la consommation moyenne en L/100km, l'année de frabrication, le type de véhicule, le nombre de cylindre, cylindree, la transmission et la boite.

- `train.csv` est l'ensemble d'entraînement
- `test.csv` est l'ensemble de test


## Chargement des données

Importation des librairies utilisées dans le calepin.

// TODO: à enlever à la fin:

Pour importer librairies:
using Pkg
Pkg.add(["CSV", "DataFrames", "Combinatorics", "Gadfly", "Distributions"], ...)

In [2655]:
# import Pkg; Pkg.add("Plots")

In [2656]:
# using CSV, DataFrames, Statistics, Dates, Gadfly, LinearAlgebra, Plots

In [2657]:
function standardize(data)
    return (data .- mean(data)) ./ std(data)
end

function standardize_data(data)
    stddata = deepcopy(data)
   for col in names(stddata)
        if eltype(stddata[!, col]) <: Number && col != "id"
            stddata[!, col] = standardize(stddata[!, col])
        end
    end
    return stddata
end

standardize_data (generic function with 1 method)

In [2658]:
function encode(data, column)
    for c in unique(data[!, column])
        data[!, Symbol(c)] = ifelse.(data[!, column] .== c, 1, 0)
    end
    return data
end

function encode_data(data)
    encoded_data = deepcopy(data)
    encoded_data = encode(encoded_data, :general_type)
    # encoded_data = encode(encoded_data, :annee)
    # encoded_data = encode(encoded_data, :type)
    encoded_data = encode(encoded_data, :transmission)
    encoded_data = encode(encoded_data, :boite)
    return encoded_data
end

encode_data (generic function with 1 method)

In [2659]:
# faire une méthode pour les outliers

# code pas tester:
# function remove_outliers(data)
#     return data[(abs.(zscore(data)) .< 3) .| isnan.(zscore(data)), :]
# end

In [2660]:
function removeRows(data)
    return select!(data, Not([:type, :transmission, :boite, :general_type,]))
    # return select!(data, Not([:annee]))
end

removeRows (generic function with 1 method)

In [2661]:
function add_rows(data)
    data[!,:volume_gaz] = data[!,:nombre_cylindres] .* data[!,:cylindree]

    # https://www.insurancenavy.com/average-car-weight/
    # https://www.auto-tests.com/fr/lightest-weight/Wagon/all/
    weight_dict = Dict("voiture_moyenne" => 3300, "VUS_petit" => 3500, "voiture_compacte" => 2800, "voiture_deux_places" => 2800, "voiture_minicompacte" => 1500, "VUS_standard" => 5000, "monospace" => 4500, "voiture_sous_compacte" => 2600, "camionnette_petit" => 4200, "break_petit" => 2640, "voiture_grande" => 4400, "camionnette_standard" => 4700, "break_moyen" => 3300)
    data[!, :weight] = [weight_dict[t] for t in data[!, :type]]

    general_type_dict = Dict("voiture_moyenne" => "voiture", "VUS_petit" => "VUS", "voiture_compacte" => "voiture", "voiture_deux_places" => "voiture", "voiture_minicompacte" => "voiture", "VUS_standard" => "VUS", "monospace" => "camionnette", "voiture_sous_compacte" => "voiture", "camionnette_petit" => "camionnette", "break_petit" => "break", "voiture_grande" => "voiture", "camionnette_standard" => "camionnette", "break_moyen" => "break")
    data[!, :general_type] = [general_type_dict[t] for t in data[!, :type]]
    
    # display(plot(data, x=:general_type, y=:consommation, color=:type))
    # println(combine(groupby(data, :type), nrow => :count))
    return data
end

add_rows (generic function with 1 method)

In [2662]:
function getStandardEncodedData(data)
    data_copy = deepcopy(data)
    standardised_data = add_rows(data_copy)
    standardised_data = standardize_data(data_copy)
    standardised_data = encode_data(standardised_data)
    standardised_data = removeRows(standardised_data)
    
    # print(standardised_data)
    return standardised_data
end




getStandardEncodedData (generic function with 1 method)

In [2663]:
function rmse(y, ychap)
    return sqrt(mean((ychap .- y).^2))
end


rmse (generic function with 1 method)

In [2664]:
function rsquared(y, ychap)
    ss_total = sum((y .- mean(y)).^2)
    ss_res = sum((y .- ychap).^2)
    return ss_res / ss_total
end


rsquared (generic function with 1 method)

In [2665]:
function split_data(data,k)
    n = size(data, 1)
    validation_size = floor(Int, n * k)

    validation_indices = rand(1:n, validation_size)  
    
    validation_data = data[validation_indices, :]
    training_data = data[setdiff(1:n, validation_indices), :]
    
    return training_data, validation_data
end

split_data (generic function with 1 method)

In [2666]:
function regression(standardised_data, trainTestSplitPercentage)
    training_data, test_data = split_data(standardised_data, trainTestSplitPercentage)
    X_train =  Matrix(training_data[:, Not(:consommation, :id)])
    y_train = training_data[:, :consommation]

    beta = X_train \ y_train

    rmseval = 0.0
    if trainTestSplitPercentage != 0.0
        X_test = Matrix(test_data[:, Not(:consommation, :id)])
        y_test = test_data[:, :consommation]
        ychap =  X_test * beta
        ychap = (ychap .* COMSOMMATION_STD) .+ COMSOMMATION_MEAN
        y_test = (y_test .* COMSOMMATION_STD) .+ COMSOMMATION_MEAN
        # print("regression\n")
        # print("r2 = ", rsquared(y_test, ychap), "\n")
        rmseval = rmse(y_test, ychap)
        # print("rmse = ", rmseval , "\n\n")
    end
    
    return rmseval, beta, training_data, test_data
end


regression (generic function with 1 method)

In [2667]:
function ridge_regression(standardised_data, lambda, trainTestSplitPercentage)
    training_data, test_data = split_data(standardised_data, trainTestSplitPercentage)
    X_train = Matrix(training_data[:, Not([:consommation, :id])])
    y_train = training_data[:, :consommation]
    beta = (X_train'X_train + lambda*I)\X_train'y_train

    rmseval = 0.0
    if trainTestSplitPercentage != 0.0
        X_test = Matrix(test_data[:, Not(:consommation, :id)])
        y_test = test_data[:, :consommation]
        ychap =  X_test * beta
        ychap = (ychap .* COMSOMMATION_STD) .+ COMSOMMATION_MEAN
        y_test = (y_test .* COMSOMMATION_STD) .+ COMSOMMATION_MEAN
        # print("ridge: \n")
        # print("r2 = ", rsquared(y_test, ychap), "\n")
        rmseval = rmse(y_test, ychap)
        # print("rmse = ", rmseval , "\n\n")
    end
    return rmseval, beta, training_data, test_data
end

ridge_regression (generic function with 1 method)

In [2668]:
# pas tester: (aucune idee si ca marche)

function svd_regression(standardised_data, trainTestSplitPercentage)
    training_data, test_data = split_data(standardised_data, trainTestSplitPercentage)
    X_train = Matrix(training_data[:, Not([:consommation, :id])])
    y_train = training_data[:, :consommation]
    
    U, S, V = svd(X_train)

    # beta = V * (Diagonal(1 ./ S) * U' * y_train)

    beta = V' * Diagonal([s > 1e-10 ? 1/s : 0 for s in S]) * U' * y_train

    rmseval = 0.0
    if trainTestSplitPercentage != 0.0
        X_test = Matrix(test_data[:, Not(:consommation,:id)])
        y_test = test_data[:, :consommation]
        ychap =  X_test * beta
        ychap = (ychap .* COMSOMMATION_STD) .+ COMSOMMATION_MEAN
        y_test = (y_test .* COMSOMMATION_STD) .+ COMSOMMATION_MEAN
        # print("regression\n")
        # print("r2 = ", rsquared(y_test, ychap), "\n")
        rmseval = rmse(y_test, ychap)
        # print("rmse = ", rmseval , "\n\n")
    end
    return rmseval,beta, training_data, test_data
end

svd_regression (generic function with 1 method)

In [2669]:
trainData = CSV.read("./data/train.csv", DataFrame)
testData = CSV.read("./data/test.csv", DataFrame)
trainData.consommation = parse.(Float64,replace.(trainData.consommation, "," => "."))
trainData.cylindree = parse.(Float64,replace.(trainData.cylindree, "," => "."))
testData.cylindree = parse.(Float64,replace.(testData.cylindree, "," => "."))
COMSOMMATION_MEAN = mean(trainData.consommation)
COMSOMMATION_STD = std(trainData.consommation)
trainData[!, :id] = 1:nrow(trainData)
testData[!, :id] = 1:nrow(testData)


1:150

In [2670]:
# trainData[!,:volume_gaz] = trainData[!,:nombre_cylindres] .* trainData[!,:cylindree]
# testData[!,:volume_gaz] = testData[!,:nombre_cylindres] .* testData[!,:cylindree]




In [2671]:
function evaluate_rmse()
    data = getStandardEncodedData(trainData)
    nrange = 1000
    n = 0
    for i in range(0, 1, length=nrange)
        n += regression(data, 0.05)[1]
    end
    print("average rmse for regression: ", n/nrange, "\n")

    n = 0
    for i in range(0, 1, length=nrange)
        n += ridge_regression(data, 0.1, 0.05)[1]
    end
    print("average rmse for ridge: ", n/nrange, "\n")

    n = 0
    for i in range(0, 1, length=nrange)
        n += svd_regression(data, 0.05)[1]
    end
    print("average rmse for svd: ", n/nrange, "\n")
end


evaluate_rmse (generic function with 1 method)

In [2672]:


function predict(data, beta)
    return (Matrix(data[:, Not(:consommation, :id)]) * beta)
end

function destandardize(data)
    return (data .* COMSOMMATION_STD) .+ COMSOMMATION_MEAN
end

function predict_and_destandardize(data, beta)
    return destandardize(predict(data, beta))
end



function find_mistakes(data_to_predict, beta, col)
    prediction = (Matrix(data_to_predict[:, Not(:consommation, :id)]) * beta)
    prediction = (prediction .* COMSOMMATION_STD) .+ COMSOMMATION_MEAN
    data_to_predict[!,:consommation] = data_to_predict[!,:consommation] .* COMSOMMATION_STD .+ COMSOMMATION_MEAN

    idtrain = 1:size(prediction, 1)

    base_data_of_predicted = innerjoin(data_to_predict, trainData, on=:id, makeunique=true)

    scatter(idtrain, prediction)
    scatter!(idtrain, base_data_of_predicted[!,:consommation])


    difference = prediction - base_data_of_predicted[!,:consommation]
    base_data_of_predicted[!, :difference] = difference
    diff_cutoff = 1
    high_diff_rows = base_data_of_predicted[abs.(difference) .> diff_cutoff, :]

    grouped_normal = combine(groupby(base_data_of_predicted, col), nrow => :count)

    grouped_high_diff = combine(groupby(high_diff_rows, col), nrow => :count)

    grouped_high_diff[!, :total_diff] = combine(groupby(high_diff_rows, col), :difference => (x -> sum(abs.(x))) => :total_diff)[:, :total_diff]

    max_diff = combine(groupby(high_diff_rows, col), :difference => (x -> maximum(abs.(x))) => :max_diff)

    grouped_high_diff = leftjoin(grouped_high_diff, max_diff, on=col)
    grouped_high_diff[!, :average_diff] = combine(groupby(high_diff_rows, col), :difference => (x -> mean(abs.(x))) => :average_abs_diff)[:, :average_abs_diff]

    grouped_high_diff[!, :rmse] = combine(groupby(high_diff_rows, col), :difference => (x -> sqrt(mean(x.^2))) => :rmse)[:, :rmse]
    grouped_normal[!, :rmse] = combine(groupby(base_data_of_predicted, col), :difference => (x -> sqrt(mean(x.^2))) => :rmse)[:, :rmse]

    percentage_high_diff = leftjoin(grouped_high_diff, grouped_normal, on=col, makeunique=true)
    percentage_high_diff[!, :percentage] = percentage_high_diff[!, :count] ./ percentage_high_diff[!, :count_1] .* 100
    percentage_high_diff = sort(percentage_high_diff, :percentage, rev=true)

    println("Percentage of data by ", col," with difference higher than 1: ")
    println(percentage_high_diff[:, [col, :percentage, :count, :count_1, :total_diff, :max_diff, :average_diff, :rmse]])

    println(grouped_normal[:, [col, :rmse]])
end




find_mistakes (generic function with 2 methods)

In [2673]:
# rmseval, betatrain, traindata, testpredictdata = regression(data, 0.2)
# find_mistakes(testpredictdata, betatrain, :annee)

In [2674]:
# beta = ridge_regression(data, 10, 0.0)[2]
function remise_regression(beta)
    test_data = getStandardEncodedData(testData)
    X_test = Matrix(test_data[!,Not(:id)])
    ychap =  X_test * beta
    ychap = (ychap .* COMSOMMATION_STD) .+ COMSOMMATION_MEAN
    remise(ychap)
end

function remise(prediction)
    id = 1:150
    df_pred = DataFrame(id=id, consommation=prediction)
    
    # using Dates
    # current_time = Dates.format(now(), "yyyy-mm-dd_HH-MM-SS")
    # file_name = "benchmark_" * current_time * ".csv"
    # CSV.write("./soumissions_potentielles/" *file_name, df_pred)
end


remise (generic function with 1 method)

In [2675]:
function main()
    data = getStandardEncodedData(trainData)
    rmseval, betatrain, traindata, testpredictdata = regression(data, 0.2)
    evaluate_rmse()
    find_mistakes(testpredictdata, betatrain, :type)
    # remise_regression(betatrain)
end
main()

average rmse for regression: 0.9301203862591678
average rmse for ridge: 0.9382209694561816
average rmse for svd: 1.99830504774732
Percentage of data by type with difference higher than 1: 
11×8 DataFrame
 Row │ type                   percentage  count  count_1  total_diff  max_diff  average_diff  rmse    
     │ String31               Float64     Int64  Int64?   Float64     Float64?  Float64       Float64 
─────┼────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ voiture_minicompacte     100.0         6        6    23.5044    6.09712       3.91741  4.3165
   2 │ monospace                100.0         2        2     4.51381   2.25997       2.2569   2.2569
   3 │ break_petit              100.0         1        1     3.51982   3.51982       3.51982  3.51982
   4 │ voiture_grande           100.0         2        2     3.25476   2.05807       1.62738  1.68341
   5 │ camionnette_standard     100.0         1        1     5.1767    5.1767    

In [2676]:
# for type in unique(trainData.type)
#     println(type)
#     data_type = trainData[trainData.type .== type, :]
#     println(combine(groupby(data_type, :transmission), :consommation => mean, :volume_gaz => mean, nrow => :nrow))
#     println()
# end

In [2677]:
# for type in unique(trainData.type)
#     println(type)
#     data_type = trainData[trainData.type .== type, :]
#     display(plot(x=data_type.volume_gaz, y=data_type.consommation))
#     println()
# end

## Exploration des données